# <span style="color:blue">First steps with `exo_k`</span>

*Author: Jeremy Leconte (CNRS/LAB/Univ. Bordeaux)*

The goal of `exo_k` is to provide a library to:

* Interpolate efficiently and easily in correlated-k and cross section tables
* Convert easily correlated-k and cross section tables from one format to another (pickle, hdf5, LMDZ GCM format).
* Adapt precomputed correlated-k tables to your need:
  * by changing the resolution
  * by changing the pressure/temperature grid
* To create tables for a mix of gases using tables for individual gases.
* Test various assumptions in an integrated radiative transfer framework.
  
This tutorial will show you how to do all that.



## Initialization

First, let's make some common initializations.   
We also import our library: `exo_k`. For this to work for the moment, you may need to put the exo_k package/directory either in the same directory as this notebook, or to add the path to your PATH.

In [7]:
import exo_k
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
plt.rcParams["figure.figsize"] = (6,3)
import time,sys,os

In [8]:
%matplotlib notebook

# <span style="color:blue">Dealing with `Ktable()` objects</span>

## Loading a `Ktable()` object

For this tutorial to work, you should launch this notebook from a base_directory that contains a `data/corrk/` directory where your correlated k files are storred.

### Setting up the search path

Our first task is to tell `exo_k` where to look for files. By default, it searches the base_directory, as can be seen by running:

In [ ]:
print(exo_k.Settings().search_path())

A new path can be added to the search path as follows

In [9]:
exo_k.Settings().add_search_path('data/xsec','data/corrk')
print(exo_k.Settings().search_path())

['/Users/jleconte/atmosphere/RadiativeTransfer/exo_k', '/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/xsec', '/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/corrk']


If you do not want to add a path, but to reset the path (in order for exo_k not to see some files in the base_directory for example), you can do it as follows:

In [ ]:
exo_k.Settings().set_search_path('data/corrk')
print(exo_k.Settings().search_path())

Notice that these methods can take as many directories as you want following one of the syntaxes below

In [ ]:
exo_k.Settings().set_search_path('.','data/xsec')
print(exo_k.Settings().search_path())
exo_k.Settings().set_search_path(*['.','data/xsec'])
print(exo_k.Settings().search_path())
exo_k.Settings().set_search_path(*('.','data/corrk'))
print(exo_k.Settings().search_path())

### Loading a Ktable from an exomol-like pickle or hdf5 file 

One of the main objects we will deal with is the `Ktable()` object. This contains a big matrix with k-coefficients for a species or a mix of species along with all the needed supporting information such as the Pressure, Temperature, wavenumber, g grids onto which the k-coefficients have been computed.  

To instentiate a `Ktable()` object from an exomol-like pickle, just run:

In [ ]:
h2o_ktab=exo_k.Ktable('H2O_R300_0.3-50mu.ktable.TauREx',
                      old_kdata_unit='cm^2',old_p_unit='bar')
print(h2o_ktab)

This has found `'data/corrk/H2O_R300_0.3-50mu.ktable.TauREx.pickle'`.

Notice that, until now and to the extent of the author's knowledge, Exomol pickle files do not contain unit information (while Exomol hdf5 files do). However, previous experience has shown us that tracking units is essential and working with SI units is better when possible. 

For this purpose, we force the user to explicitly state what are the pressure (`old_p_unit` keyword) and k-coefficient (`old_kdata_unit` keyword) units present in the file if this is not already written in there (all k-coefficients created with `exo_k` will however carry the information about units).

For example, loading this file without spelling out units results in an error.

To control the unit we want to work with, we just need to specify what units we want with the `p_unit` and `kdata_unit` keywords.

These keywords accept string input with units recognized by the `astropy.units` library such as 'Pa','mbar', and 'bar' for pressure and e.g. 'm^2' and 'cm^2' for k-coefficients. 

Note that k-coefficients are 'per molecule', but this is not a proper unit. So you do not need to specify 'm^2/molec', for example. Opacities per 'moles' or 'kg' are not supported at the moment.

So to use SI units, just reload your ktable as follows. Note that you can also force SI units to be used with the global option.
```python
exo_k.Settings().set_mks(True)
```

In [ ]:
h2o_ktab_SI=exo_k.Ktable('H2O_R300_0.3-50mu.ktable.TauREx.pickle',
                      old_kdata_unit='cm^2',old_p_unit='bar',
                      kdata_unit='m^2',p_unit='Pa')
# or simply
exo_k.Settings().set_mks(True)
h2o_ktab_SI=exo_k.Ktable('H2O_R300_0.3-50mu.ktable.TauREx',
                      old_kdata_unit='cm^2',old_p_unit='bar')
print(h2o_ktab_SI)

Notice how the pressure grid, in particular, has changed. 

### Saving a Ktable to a pickle or hdf5 file 

To save any `Ktable()` object to a pickle or hdf5 file, just use `write_pickle()` or `write_hdf5()`.
Notice that the full path (relative or absolute) must be provided, except for the file extension that will be added automatically. Here we will write an hdf5 file (more optimal than pickle)

In [ ]:
h2o_ktab_SI.write_hdf5('data/corrk/H2O_R300_0.3-50mu.ktable.SI')

This new pickle file has saved the unit information. So now, it can be loaded with only:

In [ ]:
h2o_ktab_SI=exo_k.Ktable('H2O_R300_0.3-50mu.ktable.SI')
print(h2o_ktab_SI)

### What is inside a Ktable object

To see what is inside a `Ktable()` object, you can list the attributes using `__dict__`

**Although you can access and see these attributes, they should NOT be changed manually as changing one of them usually implies that others need to be changed too. There is a method to change all the attributes that can possibly be changed.**

The most useful attributes are the following:  
 'mol', Name of the mol described  
'Ng', Number of g points  
 'filename',  Name of the file used to load the Ktable  
 'ggrid', Abscissa of the g grid  
 'weights', quadrature weights corresponding to the g points
 'kdata', Numpy array with the coeficients
 'kdata_unit', Units of the kdata  
 'pgrid', pressure grid  
 'logpgrid', Log 10 of the pressure grid  
 'Np', Number of pressure points  
 'p_unit', Units of the pressure grid  
 'tgrid', Temperature grid  
 'Nt', Number of Temperature points  
 'wns', Central points of the wavenumber bins (in cm^-1)  
 'wnedges', Edges of the wavenumber bins (in cm^-1)  
 'wls', Wavelengths (10000./wns in microns)  
 'wledges', Wavelengths (10000./wnedges in microns)  
 'Nw', Number of wavenumber points  
 'shape', shape of the kdata array  


In [ ]:
print(h2o_ktab_SI.mol)
print(h2o_ktab_SI.pgrid)

the data themselves can also be accessed directly as follows (but they cannot be modified this way):

In [ ]:
print(h2o_ktab_SI.kdata[0,0,0,:])
print(h2o_ktab_SI[0,0,0,:])

### Handling zeros

At wavelengths where data are not available, the ktable can be filled with zeros that can become a problem for some codes (e.g. because of log interpolation).  

If you want to avoid that, we have created the `remove_zeros=True` option that will replace zeros with a number 10 orders of magnitude smaller than the smallest number in the table. 
```python
h2o_ktab_SI=exo_k.Ktable(filename,remove_zeros=True,**kwargs)
```
You can also call manually the `remove_zeros()` method on a `Ktable()`object. 

### Copying a Ktable object

If for some reason, as we will do below, you need to copy a Ktable object before modifying it (to change resolution for example), you can use the `copy` method, that is a bit like the deep copy in `numpy`.

In [ ]:
new_h2o_ktab=h2o_ktab_SI.copy()
new_h2o_ktab

Sometimes it is copy only the structure and metadata, but not the big `self.kdata` table. Then, you can use:
```
new_h2o_ktab=h2o_ktab_SI.copy(cp_kdata=False)
```

## Interpolating opacities

### Basic interpolation 

To get the value of the cross section of the molecule or mix described by our `Ktable()` object, we can use:
 ```python
 self.interpolate_kdata(logp_array=None,t_array=None,log_interp=None,wngrid_limit=None)
 ```
This returns the cross section of the molecule along a LogP-T profile specified by `logp_array` and `t_array`. This returns an array of shape `(logp_array.size, self.Nw, self.Ng)`,  i.e (number of PT points, number of wavenumber points, number of g-points). `logp_array`and `t_array`must have the same size.  

It also works if `logp_array` and/or `t_array` are `floats`
    
**Notice that the input is always in Log10(Pressure) because this is the quantity used for the interpolation**

In [ ]:
logp=3.
Temp=1000.
tmp_opa=h2o_ktab_SI.interpolate_kdata(logp,Temp)
print(tmp_opa.shape)
print(tmp_opa)

In [ ]:
logp_array=np.linspace(1.,6.,6)
t_array=[1000. for logp in logp_array]
tmp_opa=h2o_ktab_SI.interpolate_kdata(logp_array,t_array)
print(tmp_opa.shape)
print(tmp_opa)

### Interpolation options

The default interpolation scheme interpolates `log(kdata)`. This can be changed directly using the `log_interp=False` keyword. But the default behavior can also be changed using `Settings().set_log_interp`. The keyword always supercedes the default behavior.

In [ ]:
logp=3.
Temp=1000.
tmp_opa_lin1=h2o_ktab_SI.interpolate_kdata(logp,Temp,log_interp=False)

In [ ]:
exo_k.Settings().set_log_interp(False)
logp=3.
Temp=1000.
tmp_opa_lin2=h2o_ktab_SI.interpolate_kdata(logp,Temp)
np.sum(tmp_opa_lin2-tmp_opa_lin1)
exo_k.Settings().set_log_interp(True) ## let's go back to default behavior

Linear interpolation is of course faster, but is believed to be less accurate. 

In [ ]:
logp=3.
Temp=1000.
%timeit h2o_ktab_SI.interpolate_kdata(logp,Temp,log_interp=True)
%timeit h2o_ktab_SI.interpolate_kdata(logp,Temp,log_interp=False)

If we just want a part of the spectrum, we can specify a **wavenumber** range (in cm^-1) to cover as follows.   
You can see that it goes much faster.

In [ ]:
logp=3.
Temp=1000.
wn_range=[6000.,5000.] # does not need to be sorted
%timeit h2o_ktab_SI.interpolate_kdata(logp,Temp,wngrid_limit=wn_range,log_interp=True)
%timeit h2o_ktab_SI.interpolate_kdata(logp,Temp,wngrid_limit=wn_range,log_interp=False)
tmp_opa=h2o_ktab_SI.interpolate_kdata(logp,Temp,wngrid_limit=wn_range)
print(tmp_opa.shape)
print(tmp_opa)

### (Log P, T) grid Remapping 

If for computational efficiency reason we just want to use a subset of the (Log P, T) grid, we can remap the table. This will modify the Table in place. So we might want to make a copy before.

In [ ]:
Small_Grid_h2o_ktab=h2o_ktab_SI.copy()
logp_array=np.arange(6.)
t_array=np.linspace(200.,1000.,9)

Small_Grid_h2o_ktab.remap_logPT(logp_array=logp_array,t_array=t_array)
Small_Grid_h2o_ktab

### Plot an opacity spectrum/distribution

To help you vizualize opacities, the  
```python
plot_spectrum(self,ax,p=1.e-5,t=200.,g=0.,logx=True,logy=True,x=1.,**kwarg)
```
method is there to make it easy to plot at the requested p,T,g point. You can also normalize the cross section 
with the volume mixing ratio of the species with the `x=vmr` keyword.

In [ ]:
%matplotlib notebook
p_plot=1.e5
t_plot=1000.
fig,axs=plt.subplots(2,2,sharex=False,sharey=False,figsize=(7,5))
h2o_ktab_SI.plot_spectrum(axs[0,0],p=p_plot,t=t_plot,g=1.,yscale='log',xscale='log')
h2o_ktab_SI.plot_spectrum(axs[0,0],p=p_plot,t=t_plot,g=0.,yscale='log',xscale='log')
h2o_ktab_SI.plot_spectrum(axs[1,0],p=p_plot,t=t_plot,g=1.,xscale='log')
h2o_ktab_SI.plot_spectrum(axs[1,0],p=p_plot,t=t_plot,g=0.,xscale='log')
h2o_ktab_SI.plot_spectrum(axs[0,1],p=p_plot,t=t_plot,g=1.,yscale='log')
h2o_ktab_SI.plot_spectrum(axs[0,1],p=p_plot,t=t_plot,g=0.,yscale='log')
h2o_ktab_SI.plot_spectrum(axs[1,1],p=p_plot,t=t_plot,g=1.)
h2o_ktab_SI.plot_spectrum(axs[1,1],p=p_plot,t=t_plot,g=0.)
#ax.set_ylim(bottom=1.e-37)
fig.tight_layout()

The g gistribution is shown with `plot_distrib`. Notice that when you ask for a log scale in g, we switch to 1-g as it is the most relevant quantity to look at in log.

In [ ]:
p_plot=1.e5
t_plot=1000.
fig,axs=plt.subplots(2,2,sharex=False,sharey=False,figsize=(7,5))
h2o_ktab_SI.plot_distrib(axs[0,0],p=p_plot,t=t_plot,wl=1.0,yscale='log',xscale='log')
h2o_ktab_SI.plot_distrib(axs[0,0],p=p_plot,t=t_plot,wl=10.,yscale='log',xscale='log')
h2o_ktab_SI.plot_distrib(axs[1,0],p=p_plot,t=t_plot,wl=1.0,xscale='log')
h2o_ktab_SI.plot_distrib(axs[1,0],p=p_plot,t=t_plot,wl=10.,xscale='log')
h2o_ktab_SI.plot_distrib(axs[0,1],p=p_plot,t=t_plot,wl=1.0,yscale='log')
h2o_ktab_SI.plot_distrib(axs[0,1],p=p_plot,t=t_plot,wl=10.,yscale='log')
h2o_ktab_SI.plot_distrib(axs[1,1],p=p_plot,t=t_plot,wl=1.0)
h2o_ktab_SI.plot_distrib(axs[1,1],p=p_plot,t=t_plot,wl=10.)
#ax.set_ylim(bottom=1.e-37)
fig.tight_layout()

## Binning down corr-k data

We now get to the core feature of `exo_k`: we want to create a new `Ktable()` object with just the spectral resolution we need in the model we want to run (whether it is a Global Climate Model, a 1D climate model, or a simple forward model for a retrieval).  

First we create a grid at a R=30 resolution. 

In [ ]:
R=10
wavenumber_grid=exo_k.wavenumber_grid_R(200.,10000.,R)
wavenumber_grid

Then we just need to use the `bin_down(wnedges)` method. However, bin_down modifies the `Ktable()` in place. So again, if we want to keep the original one in memory, we just copy it.  

`wnedges` must be a table with the edges of the new wavenumber bins. So the final Ktable will have a spectral dimension with size `wnedges.size-1`.

Optional keywords can be `weights=` and `ggrid=` where you can provide the weights you want (and in some cases the corresponding g points). In general, for an LMDZ user, just input the values inside g.dat (except for the final zero) in a `w_array`, and use `weights=w_array`.

It should only take a few seconds. (However, the bigger the final resolution wanted, the longer it takes).

In [ ]:
LowRes_h2o_ktab=h2o_ktab_SI.copy()
start=time.time()
LowRes_h2o_ktab.bin_down(wavenumber_grid)
end=time.time()
print('computation time=',end-start)
LowRes_h2o_ktab

In [ ]:
p_plot=1.e0
t_plot=300.
fig,axs=plt.subplots(1,2,sharex=False,sharey=False)
h2o_ktab_SI.plot_spectrum(axs[0],p=p_plot,t=t_plot,g=1.,yscale='log',xscale='log')
LowRes_h2o_ktab.plot_spectrum(axs[0],p=p_plot,t=t_plot,g=1.)
h2o_ktab_SI.plot_spectrum(axs[1],p=p_plot,t=t_plot,g=1.,yscale='log',xscale='log')
LowRes_h2o_ktab.plot_spectrum(axs[1],p=p_plot,t=t_plot,g=1.)
axs[1].set_xlim(left=1.,right=6.)
axs[1].set_xscale('linear')
fig.tight_layout()

## Loading and creating correlated-k tables to use with LMDZ GCM

### Creating new corr-k for LMDZ-GCM

`exo_k` has been made also to create corr-k tables to be used with the LMDZ gcm.  

To produce a LMDZ usable directory with the corr-k and all the supporting files, nothing is simpler. 

**Be careful. This section only deals with corrk *without* variable gases. For these, we need to deal with gas mixes and `Ktable5D` that are discussed a little further down.**

First, let's create a grid for our IR and VI channels and create our two ktables.

In [ ]:
R=10
IR_grid=exo_k.wavenumber_grid_R(220.,10000.,R)
VI_grid=exo_k.wavenumber_grid_R(5200.,33000.,R)

IR_h2o_ktab=h2o_ktab_SI.copy()
IR_h2o_ktab.bin_down(IR_grid)

VI_h2o_ktab=h2o_ktab_SI.copy()
VI_h2o_ktab.bin_down(VI_grid)

Once your low resolution corrk table has been produced, just run

In [ ]:
IR_h2o_ktab.write_LMDZ('data/corrk/lmdz/new_h2o_corrk',band='IR')
VI_h2o_ktab.write_LMDZ('data/corrk/lmdz/new_h2o_corrk',band='VI')

**Should you care about units?**  

Actually... no! Because we have tracked units from the beginning, and that we know LMDZ likes ktables in cm^2/molec and pressures in log mBar, **the conversion has been done automatically** for you.

The LMDZ aficionados may have noticed that in the `corrk/new_h2o_corrk` directory, two subdirectories have been created with the names `IR self.Nw` and `VI self.Nw` that contain the corr-k tables whereas we would like a single directory called `Number_of_IR_bins x Number_of_VI_bins`. Then just run:

In [ ]:
exo_k.finalize_LMDZ_dir('data/corrk/lmdz/new_h2o_corrk',IR_h2o_ktab.Nw,VI_h2o_ktab.Nw)

You just have to create a `Q.dat` file inthe directory and you are good to run LMDZ with new corrk.

### Loading existing LMDZ corr-k tables

To load a LMDZ corr-k table, the initialization method needs some more information
  * The directory to use (`path=`)
  * The resolution (`res=`)
  * Whether you want to load the 'IR' or 'VI' band (`band=`)
  
The `molecule` keyword can be useful to specify which species is described as will be seen for mixes. 

Here, if you do not specify units, you will keep the LMDZ ones (cm^2 and mBar). But we advise to use the SI units. In particular because our radiative transfer method use SI units. 

In [ ]:
IR_h2o_ktab=exo_k.Ktable(path='data/corrk/lmdz/new_h2o_corrk',res='39x19',band='IR',mol='H2O',
                kdata_unit='m^2',p_unit='Pa')
print(IR_h2o_ktab)

# <span style="color:blue">Dealing with mixes: The `Kdatabase()` object </span>

## Loading a `Kdatabase()` object

A `Kdatabase()` object mainly contains a dictionary of `Ktable()` objects identified by the name of their molecule (in the attribute `self.ktables`, but they can be accessed with `self['mol_name']`). In addition, most of the attributes of the `Ktable()` are reloaded as attributes of `Kdatabase()`.

To load a database, just specify a list of molecules and a string that is common to all your filenames.

In [ ]:
molecs=['H2O','CO2']
filt='R300_0.3-50mu.ktable.SI'
database=exo_k.Kdatabase(molecs,filt)
database['H2O']

It will look for files starting with the molecule names, followed by `_`, and containing the identifier.

If you want the moleculer name to be followed by something else than `_` (say a period), then just change the setting by typing 
```python
exo_k.Settings().set_delimiter('.')
```

Of course, this is unpractical when you do not have a homogeneous set of Ktables to start with. Then you can just feed a dictionary of molecules with the associated files path.

In [ ]:
database=exo_k.Kdatabase({'H2O':'data/corrk/H2O_R300_0.3-50mu.ktable.SI.hdf5',
                          'CO2':'data/corrk/CO2_R300_0.3-50mu.ktable.SI.pickle'})

## Create a `Ktable()` for a mix of gases

One of the main interests of databases is to actually be able to create new corr-k tables for mixtures of gases. 

Just create a dictionary with the names of the molecules and the corresponding volume mixing  ratios. Notice that the sum of the mixing ratios does not need to be equal to one. In that case, it is assumed that the rest of the gas is composed of radiatively inactive species and the opacity given is in m^2 per everage molecule (couting the inactive ones).

If a molecule is not in the database, it will just be considered radiatively inactive. 

In [ ]:
molecs=['H2O','CO2']
suffix='R300_0.3-50mu.ktable.SI'
database=exo_k.Kdatabase(molecs,suffix)

composition={'H2O':0.001,'CO2':400.e-6,'N2':'background'}

start=time.time()
mixH2O_CO2=database.create_mix_ktable(composition)

mixH2O_CO2.write_hdf5('data/corrk/mix_H2O_CO2')

end=time.time()
print("computation time ",end - start)        

Here, the `'N2':'background'`in the composition dictionary tells the code that N2 is the background 'filler' gas. So whatever the vmr given before, N2 will be adjusted to get a total vmr of 1. Because N2 is not in the database, its opacity will however not be considered. 

In [ ]:
ptest=1.e4
ttest=300.
fig,axs=plt.subplots(1,2,sharex=True,sharey=True)
for species,x in composition.items():
  if x is not 'background':
    database[species].plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,x=x,linestyle='dashed',yscale='log')
    database[species].plot_spectrum(axs[1],p=ptest,t=ttest,g=0.,x=x,linestyle='dashed',yscale='log')
mixH2O_CO2.plot_spectrum(axs[0],p=ptest,t=ttest,g=1.)
mixH2O_CO2.plot_spectrum(axs[1],p=ptest,t=ttest,g=0.)
for ax in axs :
    ax.set_xscale('log')
    ax.label_outer()
    ax.set_ylim(bottom=1.e-40)
fig.tight_layout()

## Binning down a `Kdatabase()`

Just like `Ktable()` objects, you can change the spectral resolution of an entire database with the same `bin_down(wnedges)` method. It will be applied to all ktables in place (`weights` keyword available as well). 

In [ ]:
R=10
wavenumber_grid=exo_k.wavenumber_grid_R(200.,10000.,R)
database.bin_down(wavenumber_grid)

Notice how the resolution has changed below when we run the exact same code. 

In [ ]:
ptest=1.e4
ttest=300.

fig,axs=plt.subplots(1,2,sharex=True,sharey=True)
for species,x in composition.items():
  if x is not 'background':
    database[species].plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,x=x,linestyle='dashed')
    database[species].plot_spectrum(axs[1],p=ptest,t=ttest,g=0.,x=x,linestyle='dashed')
mixH2O_CO2.plot_spectrum(axs[0],p=ptest,t=ttest,g=1.)
mixH2O_CO2.plot_spectrum(axs[1],p=ptest,t=ttest,g=0.)
for ax in axs :
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.label_outer()
    ax.set_ylim(bottom=1.e-40)
fig.tight_layout()

## Creating a LMDZ corrk with a variable gas from a `Kdatabase()`

Here, we will finally the strength of `Kdatabase` objects. Because we can create mix of gases and bin down, we have everything we need to create LMDZ like ktables for variable gases.

For this, will have to use
```python
Kdatabase.create_mix_ktable5d(vgas_comp=, bg_comp=, x_array=)
```
where:
  - `vgas_comp`must contain a dictionary of the composition of the variable gas (it can be a single molecule).
  - `bg_comp`does the same for the background gas.
  - `a_array`is an array of the vol mix ratios of the varialbe gas that we want to compute in the table. 
  
This will create a special object: `Ktable5d`that has one more dimension than a standard `Ktable`. Some usual methods can be used with them (like plotting, writing, interpolating, binning down) but they cannot be used, for example in a `Kdatabase`.

In [ ]:
start=time.time()

molecs=['H2O','CO2']
suffix='R300_0.3-50mu.ktable.SI'

database=exo_k.Kdatabase(molecs,suffix)
# we first remap on a smaller logP T grid for speed
database.remap_logPT(logp_array=np.arange(6.), t_array=[200.,300.,400.,500.,600.])
x_array=[1.e-7,0.1]
vgas={'H2O':'background'}
bg_gas={'CO2':4.e-4,'N2':'background'}
mix_var_gas=database.create_mix_ktable5d(vgas_comp=vgas,bg_comp=bg_gas,x_array=x_array)
end=time.time()
print("computation time ",end - start)        

Then we can bin down before writing to file

In [ ]:
R=10
IR_grid=exo_k.wavenumber_grid_R(220.,10000.,R)
VI_grid=exo_k.wavenumber_grid_R(5200.,33000.,R)

IR_mix_var_gas_ktab=mix_var_gas.copy()
IR_mix_var_gas_ktab.bin_down(IR_grid)

VI_mix_var_gas_ktab=mix_var_gas.copy()
VI_mix_var_gas_ktab.bin_down(VI_grid)

IR_mix_var_gas_ktab.write_LMDZ('data/corrk/lmdz/new_earth_h2o-var',band='IR')
VI_mix_var_gas_ktab.write_LMDZ('data/corrk/lmdz/new_earth_h2o-var',band='VI')

exo_k.finalize_LMDZ_dir('data/corrk/lmdz/new_earth_h2o-var',IR_mix_var_gas_ktab.Nw,VI_mix_var_gas_ktab.Nw)

Let's plot the result compared to the initial cross sections at two water mixing ratios

In [ ]:
ptest=100.
ttest=500.
fig,axs=plt.subplots(1,2,sharex=True,sharey=True)
database['H2O'].plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,x=x_array[0])
database['CO2'].plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,x=bg_gas['CO2'])
IR_mix_var_gas_ktab.plot_spectrum(axs[0],p=ptest,t=ttest,x=x_array[0],g=1.)
VI_mix_var_gas_ktab.plot_spectrum(axs[0],p=ptest,t=ttest,x=x_array[0],g=1.)
database['H2O'].plot_spectrum(axs[1],p=ptest,t=ttest,g=1.,x=x_array[-1])
database['CO2'].plot_spectrum(axs[1],p=ptest,t=ttest,g=1.,x=bg_gas['CO2'])
IR_mix_var_gas_ktab.plot_spectrum(axs[1],p=ptest,t=ttest,x=x_array[-1],g=1.)
VI_mix_var_gas_ktab.plot_spectrum(axs[1],p=ptest,t=ttest,x=x_array[-1],g=1.)
for ax in axs :
    ax.set_ylim(bottom=1.e-40)
    ax.set_yscale('log')
    ax.set_xscale('log')
fig.tight_layout()

# <span style="color:blue">Dealing with cross sections: `Xtable()`objects </span>

Cross sections are like corr-k tables, except for the fact that they do not have a "g-dimension".

Thanks to the way `numpy` handles arrays and to the `python` inheritance system, handling a `Xtable()` is very similar to handling a `Ktable()` and you can do everything that you did before.  

It is so similar, that you can even load a `Kdatabase()` filled with `Xtable()` objects. 

Just remember that the kdata array now has one less dimension. 

In [ ]:
exo_k.Settings().add_search_path('data/xsec')
xsecs=exo_k.Xtable('H2O.R10000.TauREx.pickle',
                old_kdata_unit='cm^2',old_p_unit='bar',
                kdata_unit='m^2',p_unit='Pa')
xsecs

In [ ]:
ptest=1.e2
ttest=300.
fig,ax=plt.subplots(1,1,sharex=True,sharey=False)  
xsecs.plot_spectrum(ax,p=ptest,t=ttest,yscale='log')
fig.tight_layout()

# <span style="color:blue">Creating corr-k from high resolution spectra</span>

## Dealing with `kspectrum`-like high resolution spectra

**Due to lock down, I do not have access to my cluster (and its big spectra) at the time of writing. Some recent updates to the code therefore do not have been completely tested in this area. Sorry for the inconvenience** 

Now that you spent a few cpu x hours to compute some high resolution spectra, you would like to create a corr-k matrix with it.  

For this you'll need to provide the `logpgrid` and `tgrid` arrays onto which your spectra have been computed, as well as the corresponding spectra filenames in an array of shape `(logpgrid.size, tgrid.size (, xgrid.size))`

To help you with that, we created a small function to generate the name grid from the p t (and possibly x) grid

In [ ]:
logpgrid=[1,2]
tgrid=[100.,150.,200.]
print(logpgrid,tgrid)
file_grid=exo_k.create_fname_grid('spectrum_CO2_1e{logp}Pa_{t}K.hdf5',
                  logpgrid=logpgrid,tgrid=tgrid,p_kw='logp',t_kw='t')
file_grid

In [ ]:
logpgrid=[1,2]
tgrid=[100.,150.,200.]
xgrid=[-2,-1]
print(logpgrid,tgrid)
file_grid=exo_k.create_fname_grid('spectrum_CO2_1e{logp}Pa_{t}K_lx{x}.hdf5',
                  logpgrid=logpgrid,tgrid=tgrid,xgrid=xgrid,p_kw='logp',t_kw='t',x_kw='x')
file_grid

In [ ]:
logpgrid=np.arange(5)
tgrid=np.array([100.,150.,200.])
print(logpgrid,tgrid)
file_grid=exo_k.create_fname_grid('spectrum_CO2_1e{logp}Pa_{t}K.hdf5',
                  logpgrid=logpgrid,tgrid=tgrid,p_kw='logp',t_kw='t')
file_grid

**hdf5 files are way smaller and faster to read than dat files**

To convert kspectrum files to hdf5, you can use the following function on all of your files:
```python
for path in paths_to_my_files:
    exo_k.convert_kspectrum_to_hdf5(path, path+'hdf5')
```

You now need to create an empty `Ktable()` object and fill it with the following method.
With these options, you create a table with 20 gauss point following the Gausse-Legendre quadrature. 

In [ ]:
path='your_path'

R=10
wavenumber_grid=exo_k.wavenumber_grid_R(200.,10000.,R)

start=time.time()
tmp=exo_k.Ktable()
tmp.hires_to_ktable(path=path,filename_grid=file_grid, logpgrid=logpgrid,tgrid=tgrid,
                        wnedges=wavenumber_grid, order=20, molecule='CO2')
end=time.time()
print('computation time=',end-start)

If you want to provide the g weights yourself, run this instead:
```python
path='your_path'

R=10
wavenumber_grid=exo_k.wavenumber_grid_R(200.,10000.,R)

weights=np.array(my g weights)

start=time.time()
tmp=exo_k.Ktable()
tmp.hires_to_ktable(path=path,filename_grid=file_grid, logpgrid=logpgrid,tgrid=tgrid,
                        wnedges=wavenumber_grid, weights=weights, molecule='CO2')
end=time.time()
print('computation time=',end-start)
```

## Loading `Xtable` with `kspectrum`-like spectra

You can also load your spectra to be used as cross sections.  

For this you'll need to provide the `logpgrid` and `tgrid` arrays onto which your spectra have been computed, as well as the corresponding spectra filenames in an array of shape `(logpgrid.size,tgrid.size)`

As before, create the filename grid

In [ ]:
logpgrid=np.arange(5)
tgrid=np.array([100.,150.,200.])
print(logpgrid,tgrid)
file_grid=exo_k.create_fname_grid('spectrum_CO2_1e{logp}Pa_{t}K.hdf5',
                  logpgrid=logpgrid,tgrid=tgrid,p_kw='logp',t_kw='t')
file_grid

In [ ]:
path='your path'
hires_co2_xsec=exo_k.Xtable()
hires_co2_xsec.hires_to_xsec(path=pathmascaret,filename_grid=file_grid,
                             logpgrid=logpgrid,tgrid=tgrid,
                             molecule='CO2',kdata_unit='m^2')

You can then save it to a hdf5 file with `write_hdf5()`.

# <span style="color:blue">A simple forward model to test opacities</span>

In order to have an integrated framework to test various assumptions (will my resolution be enough, are my initial data adequate, what weights should I use, etc.), we implemented a simple 1d atmosphere model able to compute the flux emitted by an atmosphere.

This revolves around the `RadAtm()` class, which defines an atmosphere with a PT profile and a composition and is linked to a `Kdatabase()` (Notice that a database can be composed of only 1 species. We however need this structure to compute the radiative transfer).

As `Xtable()` objects can be used in a `Kdatabase()`, you can do the equivalent of Line by Line radiative transfer. 

If you want to start from very high res spectra, first load them into a `Xtable()`, and then into a Database.

## Emission

In [ ]:
database=exo_k.Kdatabase(['CO2'],'R300_0.3-50mu.ktable.SI')

T0=200.
atm_ck=exo_k.RadAtm(psurf=640.e0,ptop=1.e-1,Tsurf=T0,Tstrat=100.,grav=3.4,
                    composition={'CO2':'background'},Nlev=30, kdatabase=database)

start=time.time()
FluxTop=atm_ck.emission_spectrum(integral=True,wl_range=[0.1,150.0])
SurfFlux=atm_ck.surf_bb(integral=True)
TopBB=atm_ck.top_bb(integral=True)
bintime=time.time()
print("averaging time ",bintime - start)  

The `integral=True` keyword specifies that the source function is integrated inside each bin. If `False`, the planck function is just evaluated at the center of the bin. It is faster, and ok for thin bins, but inacurate for large ones.

The `wl_range` range keyword allows you to limit the range onto wich you want the calculation to be done to save time. 

The result of `RadmAtm().emission_spectrum()` is a `Spectrum()` object onto wich some methods can be used.  For example you can compute the total flux

In [ ]:
print(FluxTop.total,'W/m^2 (BB at tsurf over the computed nu range is ',SurfFlux.total,'W/m^2)')
print('(sigma*T**4 is ',exo_k.SIG_SB*T0**4,'W/m^2)')

you can also plot and bin_down (and when spectra have the same wn grid, algeabric operations)

In [ ]:
fig,axs=plt.subplots(1,2,sharey=False)  
SurfFlux.plot_spectrum(axs[0])
TopBB.plot_spectrum(axs[0])
FluxTop.plot_spectrum(axs[0])
SurfFlux.plot_spectrum(axs[1],per_wavenumber=False)
TopBB.plot_spectrum(axs[1],per_wavenumber=False)
FluxTop.plot_spectrum(axs[1],per_wavenumber=False)

axs[0].set_ylabel('Flux ($W/m^2/cm^{-1}$)')
axs[1].set_ylabel('Flux ($W/m^2/\mu m$)')

#axs[0].invert_yaxis()
fig.tight_layout()

In [ ]:
R=100
wavenumber_grid=exo_k.wavenumber_grid_R(200.,10000.,R)
binned_spec=FluxTop.copy()
binned_spec.bin_down(wavenumber_grid)

fig,axs=plt.subplots(1,2,sharey=False)  
(SurfFlux-FluxTop).plot_spectrum(axs[0])
FluxTop.plot_spectrum(axs[1],per_wavenumber=False)
binned_spec.plot_spectrum(axs[1],per_wavenumber=False)

axs[0].set_ylabel('Flux ($W/m^2/cm^{-1}$)')
axs[1].set_ylabel('Flux ($W/m^2/\mu m$)')

#axs[0].invert_yaxis()
fig.tight_layout()

In [ ]:
origin = 'upper'
delta = 0.025

x =database.wls
y =atm_ck.play[:]
X, Y = np.meshgrid(x, y)
Z = atm_ck.exp_minus_tau()

nr, nc = Z.shape

fig, ax = plt.subplots(constrained_layout=True)
ax.invert_yaxis()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(left=0.3,right=20.)
CS = ax.contourf(X, Y, Z, 10, cmap=plt.cm.bone, origin=origin)
ax.set_title('Atm. Transmittance')
ax.set_xlabel('Wavelength (mu)')
ax.set_ylabel('Log(Pressure/Pa)')
# Make a colorbar for the ContourSet returned by the contourf call.
cbar = fig.colorbar(CS)
cbar.ax.set_ylabel('Exp(-tau)')
# Add the contour line levels to the colorbar

## Transmission

In [ ]:
T0=2000.;xH2O=1.e-7;Rs=1.*u.Rsun;grav=10.
wn_final_grid=exo_k.wavenumber_grid_R(400.,30000.,30)
database=exo_k.Kdatabase(['H2O'],'ktable*h5',remove_zeros=True)
atm_ck=exo_k.RadAtm(psurf=10.e5,ptop=1.e-4,Tsurf=T0,Tstrat=T0,grav=grav,
                    composition={'H2':'background','H2O':xH2O},Nlev=100,Rp=1.*u.Rjup,
                    kdatabase=database)
start=time.time()
spec=atm_ck.transmission_spectrum(Rstar=Rs,rayleigh=True)
spec.bin_down(wn_final_grid)
bintime=time.time()
print("averaging time ",bintime - start)  

In [ ]:
fig,axs=plt.subplots(1,2,sharex=True,figsize=(7,3))  
spec.plot_spectrum(axs[0],xscale='log')
axs[0].set_ylabel('Depth')
axs[1].set_ylabel('Depth')
fig.tight_layout()